In [171]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML

In [172]:
warehouse_input = widgets.IntText(
    value=1,
    description='Warehouse Number:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='175px', margin='0px 5px 0px 0px')
)

user_id_input = widgets.IntText(
    value=1,
    description='User ID:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100px')
)

supplier_id_input = widgets.IntText(
    value=1,
    description='Supplier ID:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='120px')
)

In [173]:
url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"


def listWarehouses():
    response = requests.get(url+"warehouses")
    warehouse_data = response.json()
    wList = []
    for w in warehouse_data:
        wList.append(w["W_ID"])
    return wList

def listUsers():
    response = requests.get(url+"users")
    user_data = response.json()
    uList = []
    for u in user_data:
        uList.append(u["U_ID"])
    return uList

def listSuppliers():
    response = requests.get(url+"suppliers")
    supplier_data = response.json()
    sList = []
    for s in supplier_data:
        sList.append(s["S_ID"])
    return sList

def validUser():
    userID = user_id_input.value
    warehouseID = warehouse_input.value
    response = requests.get(url+"users")
    user_data = response.json()
    uList = []
    for u in user_data:
        result = (
            u["U_ID"],
            u["W_ID"]
        )
        uList.append(result)
    if warehouseID not in listWarehouses():
        print("\nWarehouse not found in database...")
        return False
    elif userID not in listUsers():
        print("\nUser not found in database...")
        return False
    else:
        for user in uList:
            if userID == user[0] and warehouseID != user[1]:
                print("\nThis user does not belong in that warehouse...")
                return False
            if userID == user[0] and warehouseID == user[1]:
                return True
    
def validSupplier():
    supplierID = supplier_id_input.value
    if supplierID not in listSuppliers():
        print("\nSupplier not found in Database")
        return False
    else:
        return True

In [174]:
#Local Statistics Data Tables

url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"

def show_profit():
    print("Showing profit...")
    if validUser() == False:
        print("\nPlease choose another user or warehouse")
    else:
        user_id = user_id_input.value
        warehouse_number = warehouse_input.value
        userVal = {"user_id" : user_id}
        response = requests.post(url+ f"warehouse/{warehouse_number}/profit", json = userVal)
        profit_data = response.json()
        df = pd.DataFrame(profit_data)
        display(df)
        profit_Year = [item['profit_year'] for item in profit_data]
        profit = [item['profit'] for item in profit_data]
        plt.plot(profit_Year, profit, marker='o', color='#0D8E07', linestyle='-')
        plt.xlabel('Year')
        plt.ylabel('Profit')
        plt.title('Profit For Each Warehouse')
        plt.grid(True)
        plt.show()

def show_top_expensive_racks():
    print("Showing top 5 expensive racks...")
    if validUser() == False:
        print("\nPlease choose another user or warehouse")
    else:
        user_id = user_id_input.value
        warehouse_number = warehouse_input.value
        userVal = {"user_id" : user_id}
        response = requests.post(url+ f"warehouse/{warehouse_number}/rack/expensive", json = userVal)
        racks_data = response.json()
        df = pd.DataFrame(racks_data)
        display(df)
        
        rack_ID = []
        count = []
        for warehouse in racks_data:
            rack_ID.append(str(warehouse["RackID"]))
            count.append(warehouse["TotalProfit"])
        plt.bar(rack_ID, count, color='#0D8E07')
        plt.xlabel('Racks')
        plt.ylabel('Total Profit')
        plt.title('Top 5 most expensive racks')
        plt.show()

def show_bottom_parts_type():
    print("Showing bottom 3 parts type/material...")
    if validUser() == False:
        print("\nPlease choose another user or warehouse")
    else:
        user_id = user_id_input.value
        warehouse_number = warehouse_input.value
        userVal = {"user_id" : user_id}
        response = requests.post(url+f"warehouse/{warehouse_number}/rack/material", json = userVal)
        warehouses = response.json()
        df = pd.DataFrame(warehouses)
        display(df)
        part_type = [item['P_TYPE'] for item in warehouses]
        part_count = [item['P_Stock'] for item in warehouses]
        plt.bar(part_type, part_count, color='#0D8E07')
        plt.xlabel('Part Type')
        plt.ylabel('Total Count')
        plt.title('Bottom 3 Parts Type/Material')
        plt.show()

def show_racks_under_capacity():
    print("Showing top 5 racks under 25% capacity...")
    if validUser() == False:
        print("\nPlease choose another user or warehouse")
    else:
        user_id = user_id_input.value
        warehouse_number = warehouse_input.value
        userVal = {"user_id" : user_id}
        response = requests.post(url+f"warehouse/{warehouse_number}/rack/lowstock", json = userVal)
        warehouses = response.json()
        df = pd.DataFrame(warehouses)
        display(df)
        
        rack_ID = []
        stock = []
        for warehouse in warehouses:
            rack_ID.append(str(warehouse["RackID"]))
            stock.append(warehouse["Stock"])
        plt.bar(rack_ID, stock, color='#0D8E07')
        plt.xlabel('Rack')
        plt.ylabel('Stock%')
        plt.title('Top 5 racks under 25% capacity')
        plt.show()

def show_top_suppliers_warehouse():
    print("Showing top 3 suppliers that supplied to the warehouse...")
    if validUser() == False:
        print("\nPlease choose another user or warehouse")
    else:
        user_id = user_id_input.value
        warehouse_number = warehouse_input.value
        userVal = {"user_id" : user_id}
        response = requests.post(url+f"warehouse/{warehouse_number}/transaction/suppliers", json = userVal)
        warehouses = response.json()
        df = pd.DataFrame(warehouses)
        display(df)
        supplier_id = [str(item['s_id']) for item in warehouses]
        transaction_count = [item['transaction_count'] for item in warehouses]
        plt.bar(supplier_id, transaction_count, color='#0D8E07')
        plt.xlabel('Supplier ID')
        plt.ylabel('Transaction Count')
        plt.title('Top 3 Suppliers That Supplied to The Warehouse')
        plt.show()


def show_top_users_exchanges():
    print("Showing top 3 users that receives the most exchanges...")
    if validUser() == False:
        print("\nPlease choose another user or warehouse")
    else:
        user_id = user_id_input.value
        warehouse_number = warehouse_input.value
        userVal = {"user_id" : user_id}
        response = requests.post(url+f"warehouse/{warehouse_number}/users/receivesmost", json = userVal)
        warehouses = response.json()
        df = pd.DataFrame(warehouses)
        display(df)
        
        user_ID = []
        count = []
        for warehouse in warehouses:
            user_ID.append(str(warehouse["userID"]))
            count.append(warehouse["ExchangeCount"])
        plt.bar(user_ID, count, color='#0D8E07')
        plt.xlabel('User')
        plt.ylabel('Exchanges Received')
        plt.title('Top 3 users that received most exchanges')
        plt.show()

def show_top_days_smallest_incoming():
    print("Showing top 3 days with the smallest incoming transactions' cost...")
    if validUser() == False:
        print("\nPlease choose another user or warehouse")
    else:
        user_id = user_id_input.value
        warehouse_number = warehouse_input.value
        userVal = {"user_id" : user_id}
        response = requests.post(url+f"warehouse/{warehouse_number}/transaction/leastcost", json = userVal)
        warehouses = response.json()
        df = pd.DataFrame(warehouses)
        display(df)
        
        dates = []
        count = []
        for date in warehouses:
            dates.append(str(date["T_Date"])+'-'+str(date['T_Year']))
            count.append(date["Total_Daily_Cost"])
        plt.bar(dates, count, color='#0D8E07')
        plt.xlabel('Date')
        plt.ylabel('Total Costs')
        plt.title('Top 3 days with smallest incoming transactions')
        plt.show()

In [175]:
# Global Statistics Data Tables

url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"

def show_top_warehouses_racks():
    print("Showing top 10 warehouses with the most racks...")
    gstat = "most/rack"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)
    
    warehouse_ID = []
    count = []
    for warehouse in res:
        warehouse_ID.append(str(warehouse["W_ID"]))
        count.append(warehouse["Rack_Count"])
    plt.bar(warehouse_ID, count, color='#006969')
    plt.xlabel('Warehouses')
    plt.ylabel('Rack Count')
    plt.title('Top 10 Warehouses with most racks')
    plt.show()

def show_top_warehouses_incoming_transactions():
    print("Showing top 5 warehouse that has the most incoming transactions...")
    gstat = "most/incoming"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)
    
    warehouse_ID = []
    count = []
    for warehouse in res:
        warehouse_ID.append(str(warehouse["W_ID"]))
        count.append(warehouse["incoming_count"])
    plt.bar(warehouse_ID, count, color='#006969')
    plt.xlabel('Warehouses')
    plt.ylabel('Incoming Transactions')
    plt.title('Top 5 Warehouses with most incoming transactions')
    plt.show()

def show_top_warehouses_delivers_exchanges():
    print("Showing top 5 warehouse that delivers the most exchanges...")
    gstat = "most/deliver"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)
    
    warehouse_ID = []
    count = []
    for warehouse in res:
        warehouse_ID.append(str(warehouse["W_ID"]))
        count.append(warehouse["MostExchanges"])
    plt.bar(warehouse_ID, count, color='#006969')
    plt.xlabel('Warehouses')
    plt.ylabel('Exchanges')
    plt.title('Top 5 Warehouses that deliver most exchanges')
    plt.show()
        
def show_top_users_transactions():
    print("Showing top 3 users that made the most transactions...")
    gstat = "most/transactions"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)
    
    user_ID = []
    count = []
    for user in res:
        user_ID.append(str(user["user_ID"]))
        count.append(user["Transaction_Count"])
    plt.bar(user_ID, count, color='#006969')
    plt.xlabel('Users')
    plt.ylabel('Transactions')
    plt.title('Top 3 users with most transactions')
    plt.show()

def show_top_warehouses_least_outgoing_transactions():
    print("Showing top 3 warehouses with the least outgoing transactions...")
    gstat = "least/outgoing"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)
    
    warehouse_ID = []
    count = []
    for warehouse in res:
        warehouse_ID.append(str(warehouse["W_ID"]))
        count.append(warehouse["Outgoing_Transaction_Count"])
    plt.bar(warehouse_ID, count, color='#006969')
    plt.xlabel('Warehouses')
    plt.ylabel('Outgoing Transactions')
    plt.title('Top 3 cities with most transactions')
    plt.show()
    
def show_top_warehouses_cities():
    print("Showing top 3 warehouses cities with the most transactions...")
    gstat = "most/city"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)
    
    warehouse_ID = []
    count = []
    for warehouse in res:
        warehouse_ID.append(str(warehouse["W_City"]))
        count.append(warehouse["W_TransactionCount"])
    plt.bar(warehouse_ID, count, color='#006969')
    plt.xlabel('Cities')
    plt.ylabel('Transactions')
    plt.title('Top 3 cities with most transactions')
    plt.show()

In [176]:
#Transactions Data Tables
  
def show_latest_to_oldest():
    print("Showing latest to oldest transactions...")
    warehouse_number=warehouse_input.value
    gstat = f"transactions/warehouse/{warehouse_number}"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)
    
    dates = []
    count = []
    for date in res:
        dates.append(str(date["T_Date"])+'-'+str(date['T_Year']))
        count.append(date["T_Quantity"])
    fig, ax = plt.subplots(figsize=(15, 6)) 
    plt.bar(dates, count, color='#D5520B')
    plt.xlabel('Date')
    plt.ylabel('Amount of parts in transaction')
    plt.title('Transactions by date')
    plt.show()

In [177]:
#Prices Data Tables

url = "https://datavenge-35c4e96eb954.herokuapp.com/datavengers/"

def show_part_prices():
    print("Showing part prices...")
    gstat = "parts/price/all"
    response = requests.get(url+gstat)
    parts_data = response.json()
    part_ID = []
    part_Prices = []
    for part in parts_data:
        part_ID.append(part["P_Name"])
        part_Prices.append(part["P_Price"])
    plt.figure(figsize=(10, 5))
    plt.bar(part_ID, part_Prices, color='#5C0A41')
    plt.xlabel('Part ID')
    plt.ylabel('Price')
    plt.title('Price of Parts by Part Name')
    plt.xticks(rotation=90) 
    plt.show()

def show_parts_supplied_by_suppliers():
    print("Showing parts supplied by suppliers...")
    if validSupplier() == False:
        print("\nPlease choose another supplier...")
    else:
        supplier_id = supplier_id_input.value
        gstat = f"parts/supplier/{supplier_id}"
        response = requests.get(url+gstat)
        res = response.json()
        if res == "Not found":
            print("\nThis supplier has not supplied any parts")
        else:
            df = pd.DataFrame(res)
            display(df)

def show_parts_in_warehouse():
    print("Showing parts in warehouse...")
    warehouse_number = warehouse_input.value
    gstat = f"parts/warehouse/{warehouse_number}"
    response = requests.get(url+gstat)
    res = response.json()
    df = pd.DataFrame(res)
    display(df)
    
    part_Name = []
    part_Stock = []
    for part in res:
        part_Name.append(part["P_Name"])
        part_Stock.append(part["Stock"])
    plt.figure(figsize=(10, 5))
    plt.bar(part_Name, part_Stock, color='#5C0A41')
    plt.xlabel('Parts')
    plt.ylabel('Stock')
    plt.title('All Parts in Warehouse')
    plt.xticks(rotation=90) 
    plt.show()

In [178]:
# Dropdown Menus

local_stats_dropdown = widgets.Dropdown(
    options=['Choose', 'Profit', 'Top 5 Expensive Racks', 'Bottom 3 Parts Type/Material', 'Top 5 Racks Under 25% Capacity', 'Top 3 Suppliers Per Warehouse', 'Top 3 Users With Exchanges', 'Top 3 Days Smallest Incoming'],
    description='Local Stats:',
)

global_stats_dropdown = widgets.Dropdown(
    options=['Choose', 'Top 10 Warehouses With Most Racks', 'Top 5 Warehouses With Most Incoming Transactions', 'Top 5 Warehouses That Delivers The Most Exchanges','Top 3 Users That Made The Most Transactions','Top 3 Warehouses With The Least Outgoing Transactions', 'Top 3 Warehouses Cities With The Most Transactions'],
    description='Global Stats:',
)

transactions_dropdown = widgets.Dropdown(
    options=['Choose', 'Latest to Oldest'],
    description='Transactions:',
)

parts_dropdown = widgets.Dropdown(
    options=['Choose', 'Part Prices', 'Parts Supplied by Suppliers', 'Parts In Warehouse'],
    description='Parts:',
)

In [179]:
# Create an output widget to display results

output = widgets.Output()

def on_local_stats_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Profit':
                show_profit()
            elif change['new'] == 'Top 5 Expensive Racks':
                show_top_expensive_racks()
            elif change['new'] == 'Bottom 3 Parts Type/Material':
                show_bottom_parts_type()
            elif change['new'] == 'Top 5 Racks Under 25% Capacity':
                show_racks_under_capacity()
            elif change['new'] == 'Top 3 Suppliers Per Warehouse':
                show_top_suppliers_warehouse()
            elif change['new'] == 'Top 3 Users With Exchanges':
                show_top_users_exchanges()
            elif change['new'] == 'Top 3 Days Smallest Incoming':
                show_top_days_smallest_incoming()


def on_global_stats_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Top 10 Warehouses With Most Racks':
                show_top_warehouses_racks()
            elif change['new'] == 'Top 5 Warehouses With Most Incoming Transactions':
                show_top_warehouses_incoming_transactions()
            elif change['new'] == 'Top 5 Warehouses That Delivers The Most Exchanges':
                show_top_warehouses_delivers_exchanges()
            elif change['new'] == 'Top 3 Users That Made The Most Transactions':
                show_top_users_transactions()
            elif change['new'] == 'Top 3 Warehouses With The Least Outgoing Transactions':
                show_top_warehouses_least_outgoing_transactions()
            elif change['new'] == 'Top 3 Warehouses Cities With The Most Transactions':
                show_top_warehouses_cities()

def on_transactions_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Latest to Oldest':
                show_latest_to_oldest()

def on_parts_dropdown_change(change):
    with output:
        clear_output(wait=True)
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'] == 'Part Prices':
                show_part_prices()
            elif change['new'] == 'Parts Supplied by Suppliers':
                show_parts_supplied_by_suppliers()
            elif change['new'] == 'Parts In Warehouse':
                show_parts_in_warehouse()
                
def show_no_output():
    with output:
        clear_output(wait=False)

In [180]:
# Register the event handlers

local_stats_dropdown.observe(on_local_stats_change, names='value')
global_stats_dropdown.observe(on_global_stats_change, names='value')
transactions_dropdown.observe(on_transactions_change, names='value')
parts_dropdown.observe(on_parts_dropdown_change, names='value')


# Layout the widgets

title_widget = widgets.HTML(
    value="<h2 style='text-align: center;'>Datavengers - Phase 3</h2>"
)
reset = widgets.Button(description = 'Reset',layout=widgets.Layout(justify_content = 'flex-end'))
reset.style.button_color = '#A2B0B1'
dropdowns_vbox = widgets.HBox([local_stats_dropdown, global_stats_dropdown, transactions_dropdown, parts_dropdown],layout=widgets.Layout(justify_content='center', width='100%'))
input_box = widgets.HBox([warehouse_input, user_id_input, supplier_id_input],layout=widgets.Layout(justify_content='center', width='100%'))

# Display the main layout

display(title_widget, reset, dropdowns_vbox, input_box, output)

# Setting up reset button functionality
def resetClicked(b):
    clear_output(wait=False)
    show_no_output()
    display(title_widget, reset, dropdowns_vbox, input_box,output)
    
reset.on_click(resetClicked)

HTML(value="<h2 style='text-align: center;'>Datavengers - Phase 3</h2>")

Button(description='Reset', layout=Layout(justify_content='flex-end'), style=ButtonStyle(button_color='#A2B0B1…

Output()